# TypeCtx SRC
We will first take a script and add the Cassette logic to get the information from the stack trace.

In [1]:
using Cassette;
using DifferentialEquations: ODEProblem, solve

In [2]:
function main()
    
    # define our ode
    function sir_ode(du, u, p, t)  
        #Infected per-Capita Rate
        β = p[1]
        #Recover per-capita rate
        γ = p[2]
        #Susceptible Individuals
        S = u[1]
        #Infected by Infected Individuals
        I = u[2]

        du[1] = -β * S * I
        du[2] = β * S * I - γ * I
        du[3] = γ * I
    end

    #Pram = (Infected Per Capita Rate, Recover Per Capita Rate)
    pram = [0.1,0.05]
    #Initial Prams = (Susceptible Individuals, Infected by Infected Individuals)
    init = [0.99,0.01,0.0]
    tspan = (0.0,200.0)
    
    # create a var to our problem
    sir_prob = ODEProblem(sir_ode, init, tspan, pram)
    solution = solve(sir_prob)
    
end

main (generic function with 1 method)

In [2]:
mutable struct FCollector{I,F,C}
    depth::I
    frame::F
    data::Vector{C}
end

In [3]:
function FCollector(d::Int, f)
    FCollector(d, f, FCollector[])
end

FCollector

In [4]:
""" 
Frame(func, args, ret, subtrace)

a structure to hold metadata for recursive type information
"""
mutable struct Frame{F,T,U}
    func::F
    args::T
    ret::U
end

Frame

In [5]:
# user 
Cassette.@context TypeCtx;

In [15]:
# user
maxdepth =3
extractor = FCollector(maxdepth, Frame(nothing, (), nothing,))

FCollector{Int64,Frame{Nothing,Tuple{},Nothing},FCollector}(3, Frame{Nothing,Tuple{},Nothing}(nothing, (), nothing), FCollector[])

In [16]:
ctx = TypeCtx(metadata = extractor);

In [8]:
# add boilerplate for functionality
function Cassette.overdub(ctx::TypeCtx, f, args...)
    c = FCollector(ctx.metadata.depth-1, Frame(f, args, Any))
    push!(ctx.metadata.data, c)
    if c.depth > 0 && Cassette.canrecurse(ctx, f, args...)
        newctx = Cassette.similarcontext(ctx, metadata = c)
        z = Cassette.recurse(newctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    else
        z = Cassette.fallback(ctx, f, args...)
        c.frame.ret = typeof(z)
        return z
    end
end

In [9]:
Cassette.canrecurse(ctx::TypeCtx,::typeof(ODEProblem), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Base.vect), args...) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(FCollector)) = false
Cassette.canrecurse(ctx::TypeCtx,::typeof(Frame)) = false

In [17]:
Cassette.overdub(ctx,main)

retcode: Success
Interpolation: 1st order linear
t: 2001-element Array{Float64,1}:
   0.0
   0.1
   0.2
   0.3
   0.4
   0.5
   0.6
   0.7
   0.8
   0.9
   1.0
   1.1
   1.2
   ⋮  
 198.9
 199.0
 199.1
 199.2
 199.3
 199.4
 199.5
 199.6
 199.7
 199.8
 199.9
 200.0
u: 2001-element Array{Array{Float64,1},1}:
 [0.99, 0.01, 0.0]                 
 [0.989901, 0.0100491, 5.01227e-5] 
 [0.989801, 0.0100985, 0.000100492]
 [0.989701, 0.010148, 0.000151108] 
 [0.9896, 0.0101979, 0.000201972]  
 [0.989499, 0.0102479, 0.000253087]
 [0.989397, 0.0102982, 0.000304452]
 [0.989295, 0.0103487, 0.000356069]
 [0.989193, 0.0103995, 0.000407939]
 [0.989089, 0.0104504, 0.000460064]
 [0.988986, 0.0105017, 0.000512444]
 [0.988882, 0.0105531, 0.000565081]
 [0.988777, 0.0106049, 0.000617976]
 ⋮                                 
 [0.21018, 0.0149422, 0.774878]    
 [0.210148, 0.014899, 0.774953]    
 [0.210117, 0.0148558, 0.775027]   
 [0.210086, 0.0148128, 0.775101]   
 [0.210055, 0.01477, 0.775175]     
 [0.2100

In [18]:
main() = include("SIRModel.jl")

main (generic function with 1 method)

In [19]:
Cassette.overdub(ctx,main);

In [7]:
using LightGraphs;
using MetaGraphs;

In [13]:
g = MetaDiGraph()
function build_graph(collector::FCollector)
    add_vertex!(g,:name,collector.frame.args)
    add_vertex!(g,:name,collector.frame.ret)
    add_edge!(g,nv(g)-1,nv(g),:name,collector.frame.func)
    for frame in collector.data
        build_graph(frame)
    end
    return g
end

build_graph (generic function with 1 method)

In [14]:
build_graph(extractor)

{8, 4} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

# Test

In [1]:
using Cassette
using SemanticModels.Dubstep

┌ Info: Precompiling SemanticModels [88974b6f-63f0-5f7a-882c-b779d2960b8e]
└ @ Base loading.jl:1192


loaded


In [2]:
sir_path = "SIRModel.jl"

"SIRModel.jl"

In [3]:
ext = typegraph(sir_path)

{6, 3} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

# Graph

In [4]:
function parsefile(path, modprefix="Modeling")
    s = read(path, String)
    try
        expr = Meta.parse(s)
        return expr
    catch
        s = "module $modprefix\n$s \nend"
        expr = Meta.parse(s)
        return expr
    end
end

parsefile (generic function with 2 methods)

In [21]:
s = read("SIRModel.jl",String)
ast = Meta.parse(s)
eval(ast)
sir.main()

retcode: Success
Interpolation: 1st order linear
t: 2001-element Array{Float64,1}:
   0.0
   0.1
   0.2
   0.3
   0.4
   0.5
   0.6
   0.7
   0.8
   0.9
   1.0
   1.1
   1.2
   ⋮  
 198.9
 199.0
 199.1
 199.2
 199.3
 199.4
 199.5
 199.6
 199.7
 199.8
 199.9
 200.0
u: 2001-element Array{Array{Float64,1},1}:
 [0.99, 0.01, 0.0]                 
 [0.989901, 0.0100491, 5.01227e-5] 
 [0.989801, 0.0100985, 0.000100492]
 [0.989701, 0.010148, 0.000151108] 
 [0.9896, 0.0101979, 0.000201972]  
 [0.989499, 0.0102479, 0.000253087]
 [0.989397, 0.0102982, 0.000304452]
 [0.989295, 0.0103487, 0.000356069]
 [0.989193, 0.0103995, 0.000407939]
 [0.989089, 0.0104504, 0.000460064]
 [0.988986, 0.0105017, 0.000512444]
 [0.988882, 0.0105531, 0.000565081]
 [0.988777, 0.0106049, 0.000617976]
 ⋮                                 
 [0.21018, 0.0149422, 0.774878]    
 [0.210148, 0.014899, 0.774953]    
 [0.210117, 0.0148558, 0.775027]   
 [0.210086, 0.0148128, 0.775101]   
 [0.210055, 0.01477, 0.775175]     
 [0.2100

In [13]:
f = open("SIRModel.jl")
s = read(f,String)
ast = Meta.parse(s)

:(module sir
  #= none:1 =#
  #= none:2 =#
  using DifferentialEquations
  #= none:4 =#
  function sir_ode(du, u, p, t)
      #= none:6 =#
      β = p[1]
      #= none:8 =#
      γ = p[2]
      #= none:11 =#
      S = u[1]
      #= none:13 =#
      I = u[2]
      #= none:15 =#
      du[1] = -β * S * I
      #= none:16 =#
      du[2] = β * S * I - γ * I
      #= none:17 =#
      du[3] = γ * I
  end
  #= none:21 =#
  pram = [0.1, 0.05]
  #= none:23 =#
  init = [0.99, 0.01, 0.0]
  #= none:24 =#
  tspan = (0.0, 200.0)
  #= none:26 =#
  sir_prob = ODEProblem(sir_ode, init, tspan, pram)
  #= none:27 =#
  sir_sol = solve(sir_prob, saveat=0.1)
  #= none:30 =#
  using Plots
  #= none:31 =#
  plot(sir_sol, xlabel="Time", ylabel="Number")
  #= none:33 =#
  function sir_ode2(du, u, p, t)
      #= none:34 =#
      (S, I, R) = u
      #= none:35 =#
      (b, g) = p
      #= none:36 =#
      du[1] = -b * S * I
      #= none:37 =#
      du[2] = b * S * I - g * I
      #= none:38 =#
      du[3] = g * I